In [ ]:
import numpy as np
from scipy.optimize import fsolve, minimize_scalar
import re


# -------------------------------------
k_B_j = 1.380649e-23
M_taylor = 2.75
E_dipole_sum = (0.6 + 0.9) * 1.602e-19 / 1e-10
v0 = 5e12

a_pure_bcc_comprehensive = {
    'Mo': 3.14e-10, 'Nb': 3.30e-10, 'Ta': 3.30e-10, 'W':  3.165e-10,
    'Ti': 3.31e-10, 'V':  3.05e-10, 'Zr': 3.58e-10, 'Hf': 3.55e-10,
}

E_wi_dict_comprehensive = {
    'Mo': 0.027, 'Nb': -0.033, 'Ta': -0.081, 'W':  0.0814, 'Ti': -0.0813,
    'V':  0.018, 'Zr': -0.169, 'Hf': -0.016,
}




# ---------------------------------------------
def parse_composition(alloy_formula: str):
    elements = re.findall('[A-Z][a-z]?', alloy_formula)
    return {elem: 1.0 / len(elements) for elem in elements}

def predict_yield_strength(composition_dict: dict, e_wi_database: dict, T: float, strain_rate: float):
    a_alloy = sum(fraction * a_pure_bcc_comprehensive[elem]
                  for elem, fraction in composition_dict.items() if elem in a_pure_bcc_comprehensive)
    b = (np.sqrt(3) / 2) * a_alloy
    ap = 0.942 * b
    Ak = 10 * b
    E_wi_joules = {elem: val * 1.60218e-19 for elem, val in e_wi_database.items()}
    var_E_sq = sum(composition_dict[elem] * (E_wi_joules[elem]**2)
                   for elem in composition_dict if elem in E_wi_joules)
    if var_E_sq == 0: return 0.0, a_alloy
    R_param = 27 * var_E_sq**2 / (ap**4 * b**6 * Ak**2)
    def calculate_tau_y(L):
        tau_j = E_dipole_sum / (4 * b * L)
        def equation_for_tau_k(tau_k_guess):
            tau_k = tau_k_guess[0];
            if tau_k <= 0: return 1e12
            delta_V = (3*var_E_sq/(2*tau_k*ap*b**2)) + (tau_k**2*ap**3*b**4*Ak**2/(6*var_E_sq))
            log_term = np.log((5*np.pi*k_B_j*T)**2 * v0 * ap * b / (delta_V**2 * strain_rate))
            S_param = (18*k_B_j*T*var_E_sq / (ap**3*b**4*Ak**2)) * log_term
            return tau_k**4 + S_param * tau_k - R_param
        tau_k_initial_guess = (R_param)**0.25
        tau_k_solution, _, success, _ = fsolve(equation_for_tau_k, [tau_k_initial_guess], full_output=True)
        return (tau_k_solution[0] + tau_j) if success and tau_k_solution[0] > 0 else np.inf
    opt_result = minimize_scalar(calculate_tau_y, bounds=(10*b, 1000*b), method='bounded')
    return (M_taylor * opt_result.fun) / 1e9, a_alloy


# -----------------------
def main():
    T = 298.15 # 25°C
    strain_rate = 1e-3
    alloys_to_predict = {
        'MoNbTaW': 1.05,
        'TiZrHfNbTa': 1.08,
        'HfNbTiZrV': 1.10,
        'MoNbTaWV': 1.25,
        'MoNbTiZr': 1.55,
    }
    print(f"预测温度: {T - 273.15:.1f} °C, 使用综合参数集")
    print("=" * 80)
    print(f"{'Alloy':<12} | {'Experimental (GPa)':<20} | {'Model Prediction (GPa)':<24} | {'Error':<10}")
    print("-" * 80)

    for formula, exp_strength in alloys_to_predict.items():
        composition = parse_composition(formula)
        pred_strength, _ = predict_yield_strength(composition, E_wi_dict_comprehensive, T, strain_rate)
        error = (pred_strength - exp_strength) / exp_strength * 100
        print(f"{formula:<12} | {exp_strength:<20.3f} | {pred_strength:<24.3f} | {error:<10.1f}%")

# --- 4. 程序入口 ---
# -------------------
if __name__ == "__main__":
    main()

预测温度: 25.0 °C, 使用综合参数集
Alloy        | Experimental (GPa)   | Model Prediction (GPa)   | Error     
--------------------------------------------------------------------------------
HfNbTiZr     | 0.880                | 1.488                    | 69.1      %
MoNbTaW      | 1.050                | 1.219                    | 16.1      %
TiZrHfNbTa   | 1.080                | 1.486                    | 37.6      %
HfNbTiZrV    | 1.100                | 1.450                    | 31.8      %
MoNbTaWV     | 1.250                | 1.155                    | -7.6      %
MoNbTiZr     | 1.550                | 1.640                    | 5.8       %
